In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('data/gemstone.csv')

In [3]:
# seggregate independent and dependent variable
X = df.drop('price', axis=1)
y = df['price']

In [4]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.30, random_state=30)

In [5]:
# pipline

num_cols = X.select_dtypes(exclude='object').columns
cat_cols = X.select_dtypes(include='object').columns

In [6]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [7]:
from sklearn.impute import SimpleImputer # handling missing values
from sklearn.preprocessing import StandardScaler # scaling data
from sklearn.preprocessing import OrdinalEncoder # ranking data
#pipline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [8]:
num_pipline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

cat_pipline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinal',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('sclaer',StandardScaler())
    ]
)

preprocessor = ColumnTransformer([
    ('num_pipline',num_pipline,num_cols),
    ('cat_pipline',cat_pipline,cat_cols)
])

In [9]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [10]:
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [12]:

# Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1013.904802565844
MAE: 674.0257224619529
R2 score 93.68908132630604


Lasso
Model Training Performance
RMSE: 1013.8784227049385
MAE: 675.0716918421101
R2 score 93.68940971806553


Ridge
Model Training Performance
RMSE: 1013.9060209095579
MAE: 674.055793609431
R2 score 93.68906615945271


Elasticnet
Model Training Performance
RMSE: 1533.4194703386743
MAE: 1060.742542700525
R2 score 85.56488759855311


